In [ ]:
# Installing psycopg2 shall fail so use psycopg2-binary
!pip install psycopg2-binary

In [1]:
# Important imports
import psycopg2
import pandas as pd
import numpy as np

In [18]:
# Establish connection with the database by providing credentials and database
# information.

pg_conn = psycopg2.connect(
    database="",
    user="",
    password="",
    host="",
    port=''
)

pg_conn

In [3]:
# Get the data from github link DMA uploaded and check for data consistency
df = pd.read_csv("https://raw.githubusercontent.com/labs12-should-i-live-here/DS/manjula-mishra/noaa_third.csv")

In [4]:
# Check the first few rows
df.head()

,year,FIPS,Winter Weather,Storm,Flood,Fire,Heat,Drought,Tornado,Hurricane,earthquakes
0,1996,1001,3,6,2,0,1,0,0,0,0
1,1996,1003,0,18,2,0,0,0,0,0,0
2,1996,1005,2,1,0,0,1,0,0,0,0
3,1996,1007,4,5,2,0,1,0,0,0,0
4,1996,1009,4,17,0,0,1,0,0,0,0


In [5]:
# Check for any misisng values
df.isna().sum()

year              0
FIPS              0
Winter Weather    0
Storm             0
Flood             0
Fire              0
Heat              0
Drought           0
Tornado           0
Hurricane         0
earthquakes       0
dtype: int64

In [6]:
# Check the shape
df.shape

(72243, 11)

In [7]:
# Check the data types
df.dtypes

year              int64
FIPS              int64
Winter Weather    int64
Storm             int64
Flood             int64
Fire              int64
Heat              int64
Drought           int64
Tornado           int64
Hurricane         int64
earthquakes       int64
dtype: object

In [8]:
# Define the template of the table operation.
drop_noaa_table = """DROP TABLE IF EXISTS noaa_historical;"""

# no primary key assigned since year or/and fipscode both
# repeat.
create_noaa_table = """ CREATE TABLE noaa_historical(
    year int,
    fipscode int,
    winterweather int,
    storm int,
    flood int,
    fire int,
    heat int,
    drought int,
    tornado int,
    hurricane int,
    earthquake int
);"""

# Get count of number of records in the table
get_count_noaa_table = """SELECT count(*) FROM noaa_historical;"""

In [9]:
pg_conn.rollback()

In [10]:
# Create the table and commit
pg_cur = pg_conn.cursor()

In [11]:
pg_cur.execute(drop_noaa_table)
pg_cur.execute(create_noaa_table)

In [12]:
pg_conn.commit()

In [13]:
pg_cur.execute(get_count_noaa_table)
count = pg_cur.fetchone()
print(count)

(0,)


In [13]:
# Get the csv file locally for loading
!wget https://raw.githubusercontent.com/labs12-should-i-live-here/DS/manjula-mishra/noaa_third.csv

--2019-05-09 08:56:54--  https://raw.githubusercontent.com/labs12-should-i-live-here/DS/manjula-mishra/noaa_third.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.24.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.24.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2128256 (2.0M) [text/plain]
Saving to: ‘noaa_third.csv.1’

noaa_third.csv.1    100%[===================>]   2.03M  6.41MB/s    in 0.3s    

2019-05-09 08:56:54 (6.41 MB/s) - ‘noaa_third.csv.1’ saved [2128256/2128256]



In [14]:
with open('noaa_third.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f)  # Skip the header row.
    pg_cur.copy_from(f, 'noaa_historical', sep=',')

pg_conn.commit()

In [15]:
# Check the row count in noaa_historical after insert
pg_cur.execute(get_count_noaa_table)
count = pg_cur.fetchone()
print(count)

(72243,)


In [16]:
# Sample select testing 1
get_all_records_in_2000 = """ SELECT * FROM noaa_historical
WHERE year=2000;
"""
pg_cur.execute(get_all_records_in_2000)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2000, 1001, 1, 15, 1, 0, 0, 0, 1, 0, 0)
(2000, 1003, 0, 18, 0, 0, 0, 0, 4, 0, 0)
(2000, 1005, 0, 3, 0, 0, 0, 0, 0, 0, 0)
(2000, 1007, 1, 9, 0, 0, 0, 0, 1, 0, 0)
(2000, 1009, 1, 9, 1, 0, 0, 0, 0, 0, 0)
(2000, 1011, 0, 2, 0, 0, 0, 0, 3, 0, 0)
(2000, 1013, 1, 5, 0, 0, 1, 0, 0, 0, 0)
(2000, 1015, 1, 7, 0, 0, 0, 0, 2, 0, 0)
(2000, 1017, 1, 1, 0, 0, 0, 0, 0, 0, 0)
(2000, 1019, 2, 3, 0, 0, 0, 0, 1, 0, 0)
(2000, 1021, 1, 8, 0, 0, 0, 0, 1, 0, 0)
(2000, 1023, 2, 13, 1, 0, 1, 0, 1, 0, 0)
(2000, 1025, 2, 12, 0, 0, 1, 0, 0, 0, 0)
(2000, 1027, 1, 9, 0, 0, 0, 0, 0, 0, 0)
(2000, 1029, 1, 6, 0, 0, 0, 0, 1, 0, 0)
(2000, 1031, 0, 9, 0, 0, 0, 0, 1, 0, 0)
(2000, 1033, 1, 17, 0, 0, 0, 0, 0, 0, 0)
(2000, 1035, 2, 7, 0, 0, 1, 0, 0, 0, 0)
(2000, 1037, 1, 3, 0, 0, 0, 0, 1, 0, 0)
(2000, 1039, 1, 16, 0, 0, 1, 0, 0, 0, 0)
(2000, 1041, 1, 5, 0, 0, 1, 0, 0, 0, 0)
(2000, 1043, 1, 29, 2, 0, 0, 0, 0, 0, 0)
(2000, 1045, 0, 7, 0, 0, 0, 0, 1, 0, 0)
(2000, 1047, 0, 8, 0, 0, 0, 0, 0, 0, 0)
(2000, 1049, 2, 17, 0, 0, 0, 0, 0

(2000, 31055, 6, 18, 1, 0, 0, 0, 0, 0, 0)
(2000, 31057, 1, 17, 0, 0, 0, 4, 0, 0, 0)
(2000, 31059, 5, 5, 0, 0, 0, 1, 1, 0, 0)
(2000, 31061, 6, 6, 2, 0, 0, 1, 0, 0, 0)
(2000, 31063, 1, 21, 2, 0, 0, 0, 3, 0, 0)
(2000, 31065, 6, 12, 0, 0, 0, 1, 0, 0, 0)
(2000, 31067, 4, 8, 0, 0, 0, 0, 0, 0, 0)
(2000, 31069, 0, 15, 0, 0, 0, 0, 0, 0, 0)
(2000, 31071, 1, 21, 0, 0, 0, 0, 0, 0, 0)
(2000, 31073, 5, 7, 0, 0, 0, 1, 1, 0, 0)
(2000, 31075, 1, 8, 0, 0, 0, 0, 0, 0, 0)
(2000, 31077, 4, 5, 0, 0, 0, 1, 0, 0, 0)
(2000, 31079, 6, 6, 0, 0, 0, 1, 0, 0, 0)
(2000, 31081, 6, 7, 0, 0, 0, 1, 1, 0, 0)
(2000, 31083, 6, 10, 0, 0, 0, 1, 0, 0, 0)
(2000, 31085, 1, 11, 0, 0, 0, 0, 1, 0, 0)
(2000, 31087, 2, 14, 0, 0, 0, 4, 0, 0, 0)
(2000, 31089, 1, 16, 1, 0, 0, 0, 3, 0, 0)
(2000, 31091, 1, 15, 0, 0, 0, 0, 2, 0, 0)
(2000, 31093, 5, 9, 0, 0, 0, 1, 0, 0, 0)
(2000, 31095, 4, 8, 1, 0, 0, 0, 0, 0, 0)
(2000, 31097, 4, 3, 0, 0, 0, 0, 0, 0, 0)
(2000, 31099, 7, 7, 1, 0, 0, 1, 0, 0, 0)
(2000, 31101, 1, 16, 0, 0, 0, 0, 0, 0, 0)
(200

In [17]:
# Sample select testing 2
get_all_records_between_2000_2005 = """ SELECT * FROM noaa_historical
WHERE year BETWEEN 2000 AND 2005;
"""
pg_cur.execute(get_all_records_between_2000_2005)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2000, 1001, 1, 15, 1, 0, 0, 0, 1, 0, 0)
(2000, 1003, 0, 18, 0, 0, 0, 0, 4, 0, 0)
(2000, 1005, 0, 3, 0, 0, 0, 0, 0, 0, 0)
(2000, 1007, 1, 9, 0, 0, 0, 0, 1, 0, 0)
(2000, 1009, 1, 9, 1, 0, 0, 0, 0, 0, 0)
(2000, 1011, 0, 2, 0, 0, 0, 0, 3, 0, 0)
(2000, 1013, 1, 5, 0, 0, 1, 0, 0, 0, 0)
(2000, 1015, 1, 7, 0, 0, 0, 0, 2, 0, 0)
(2000, 1017, 1, 1, 0, 0, 0, 0, 0, 0, 0)
(2000, 1019, 2, 3, 0, 0, 0, 0, 1, 0, 0)
(2000, 1021, 1, 8, 0, 0, 0, 0, 1, 0, 0)
(2000, 1023, 2, 13, 1, 0, 1, 0, 1, 0, 0)
(2000, 1025, 2, 12, 0, 0, 1, 0, 0, 0, 0)
(2000, 1027, 1, 9, 0, 0, 0, 0, 0, 0, 0)
(2000, 1029, 1, 6, 0, 0, 0, 0, 1, 0, 0)
(2000, 1031, 0, 9, 0, 0, 0, 0, 1, 0, 0)
(2000, 1033, 1, 17, 0, 0, 0, 0, 0, 0, 0)
(2000, 1035, 2, 7, 0, 0, 1, 0, 0, 0, 0)
(2000, 1037, 1, 3, 0, 0, 0, 0, 1, 0, 0)
(2000, 1039, 1, 16, 0, 0, 1, 0, 0, 0, 0)
(2000, 1041, 1, 5, 0, 0, 1, 0, 0, 0, 0)
(2000, 1043, 1, 29, 2, 0, 0, 0, 0, 0, 0)
(2000, 1045, 0, 7, 0, 0, 0, 0, 1, 0, 0)
(2000, 1047, 0, 8, 0, 0, 0, 0, 0, 0, 0)
(2000, 1049, 2, 17, 0, 0, 0, 0, 0

(2000, 18127, 4, 6, 1, 0, 0, 0, 1, 0, 0)
(2000, 18129, 7, 7, 1, 0, 0, 0, 0, 0, 0)
(2000, 18131, 0, 9, 0, 0, 0, 0, 0, 0, 0)
(2000, 18133, 2, 4, 4, 0, 0, 0, 0, 0, 0)
(2000, 18135, 1, 4, 3, 0, 0, 0, 0, 0, 0)
(2000, 18137, 1, 7, 4, 0, 0, 0, 0, 0, 0)
(2000, 18139, 1, 7, 1, 0, 0, 0, 0, 0, 0)
(2000, 18141, 4, 15, 0, 0, 0, 0, 0, 0, 0)
(2000, 18143, 0, 6, 2, 0, 0, 0, 0, 0, 0)
(2000, 18145, 0, 6, 2, 0, 0, 0, 0, 0, 0)
(2000, 18147, 7, 3, 3, 0, 0, 0, 0, 0, 0)
(2000, 18149, 1, 13, 0, 0, 0, 0, 0, 0, 0)
(2000, 18151, 2, 11, 0, 0, 0, 0, 0, 0, 0)
(2000, 18153, 2, 5, 4, 0, 0, 0, 0, 0, 0)
(2000, 18155, 1, 5, 3, 0, 0, 0, 0, 0, 0)
(2000, 18157, 2, 6, 4, 0, 0, 0, 0, 0, 0)
(2000, 18159, 3, 4, 0, 0, 0, 0, 0, 0, 0)
(2000, 18161, 1, 8, 2, 0, 0, 0, 0, 0, 0)
(2000, 18163, 7, 16, 3, 0, 0, 0, 0, 0, 0)
(2000, 18165, 2, 7, 2, 0, 0, 0, 0, 0, 0)
(2000, 18167, 2, 3, 3, 0, 0, 0, 0, 0, 0)
(2000, 18169, 3, 12, 0, 0, 0, 0, 2, 0, 0)
(2000, 18171, 2, 4, 2, 0, 0, 0, 0, 0, 0)
(2000, 18173, 7, 6, 4, 0, 0, 0, 0, 0, 0)
(2000, 1817

(2000, 29143, 7, 4, 3, 0, 0, 0, 1, 0, 0)
(2000, 29145, 3, 7, 3, 0, 2, 2, 0, 0, 0)
(2000, 29147, 3, 14, 1, 0, 2, 1, 1, 0, 0)
(2000, 29149, 4, 9, 0, 0, 2, 2, 0, 0, 0)
(2000, 29151, 4, 4, 1, 0, 0, 0, 0, 0, 0)
(2000, 29153, 4, 14, 1, 0, 2, 2, 0, 0, 0)
(2000, 29155, 1, 10, 0, 0, 0, 0, 0, 0, 0)
(2000, 29157, 6, 3, 0, 0, 0, 0, 0, 0, 0)
(2000, 29159, 6, 17, 2, 0, 2, 1, 0, 0, 0)
(2000, 29161, 3, 4, 1, 1, 2, 2, 0, 0, 0)
(2000, 29163, 4, 16, 0, 0, 0, 0, 1, 0, 0)
(2000, 29165, 5, 17, 1, 0, 4, 1, 0, 0, 0)
(2000, 29167, 2, 7, 2, 1, 2, 2, 1, 0, 0)
(2000, 29169, 3, 8, 0, 0, 2, 2, 0, 0, 0)
(2000, 29171, 3, 6, 0, 0, 2, 1, 0, 0, 0)
(2000, 29173, 4, 9, 0, 0, 0, 0, 1, 0, 0)
(2000, 29175, 4, 10, 0, 0, 2, 1, 0, 0, 0)
(2000, 29177, 4, 15, 0, 0, 2, 1, 0, 0, 0)
(2000, 29179, 3, 1, 0, 0, 0, 0, 0, 0, 0)
(2000, 29181, 7, 5, 1, 0, 0, 0, 0, 0, 0)
(2000, 29183, 5, 27, 2, 0, 2, 0, 0, 0, 0)
(2000, 29185, 2, 11, 2, 1, 2, 2, 0, 0, 0)
(2000, 29186, 3, 2, 0, 0, 0, 0, 0, 0, 0)
(2000, 29187, 3, 1, 0, 0, 1, 0, 0, 0, 0)
(2000,

(2000, 47061, 0, 7, 1, 0, 0, 0, 0, 0, 0)
(2000, 47063, 3, 11, 0, 0, 0, 0, 0, 0, 0)
(2000, 47065, 3, 18, 3, 0, 0, 0, 0, 0, 0)
(2000, 47067, 3, 8, 0, 0, 0, 0, 0, 0, 0)
(2000, 47069, 1, 8, 0, 0, 0, 0, 0, 0, 0)
(2000, 47071, 1, 3, 0, 0, 0, 0, 0, 0, 0)
(2000, 47073, 3, 9, 0, 0, 0, 0, 0, 0, 0)
(2000, 47075, 2, 3, 0, 0, 0, 0, 0, 0, 0)
(2000, 47077, 1, 9, 0, 0, 0, 0, 0, 0, 0)
(2000, 47079, 0, 10, 0, 0, 0, 0, 0, 0, 0)
(2000, 47081, 1, 5, 0, 0, 0, 0, 1, 0, 0)
(2000, 47083, 1, 4, 0, 0, 0, 0, 1, 0, 0)
(2000, 47085, 0, 9, 0, 0, 0, 0, 0, 0, 0)
(2000, 47087, 2, 2, 1, 0, 0, 0, 0, 0, 0)
(2000, 47089, 3, 11, 0, 0, 0, 0, 0, 0, 0)
(2000, 47091, 4, 5, 0, 0, 0, 0, 0, 0, 0)
(2000, 47093, 3, 17, 1, 0, 0, 0, 1, 0, 0)
(2000, 47095, 2, 4, 0, 0, 0, 0, 0, 0, 0)
(2000, 47097, 2, 6, 0, 0, 0, 0, 0, 0, 0)
(2000, 47099, 0, 10, 7, 0, 0, 0, 2, 0, 0)
(2000, 47101, 0, 5, 0, 0, 0, 0, 0, 0, 0)
(2000, 47103, 0, 6, 5, 0, 0, 0, 1, 0, 0)
(2000, 47105, 3, 4, 0, 0, 0, 0, 0, 0, 0)
(2000, 47107, 3, 6, 0, 0, 0, 0, 0, 0, 0)
(2000, 471

(2000, 55093, 2, 7, 0, 0, 0, 0, 0, 0, 0)
(2000, 55095, 1, 6, 0, 0, 0, 0, 2, 0, 0)
(2000, 55097, 2, 9, 1, 0, 0, 0, 0, 0, 0)
(2000, 55099, 1, 6, 0, 0, 0, 0, 0, 0, 0)
(2000, 55101, 4, 10, 7, 0, 0, 0, 1, 0, 0)
(2000, 55103, 2, 6, 3, 0, 0, 0, 0, 0, 0)
(2000, 55105, 4, 19, 3, 0, 1, 0, 1, 0, 0)
(2000, 55107, 0, 18, 1, 0, 0, 0, 0, 0, 0)
(2000, 55109, 2, 10, 0, 0, 0, 0, 1, 0, 0)
(2000, 55111, 3, 11, 3, 0, 0, 0, 0, 0, 0)
(2000, 55113, 1, 5, 0, 0, 0, 0, 0, 0, 0)
(2000, 55115, 2, 5, 1, 0, 0, 0, 0, 0, 0)
(2000, 55117, 4, 5, 1, 0, 0, 0, 0, 0, 0)
(2000, 55119, 1, 10, 1, 0, 0, 0, 0, 0, 0)
(2000, 55121, 4, 10, 1, 0, 0, 0, 0, 0, 0)
(2000, 55123, 3, 8, 4, 0, 0, 0, 1, 0, 0)
(2000, 55125, 7, 8, 2, 0, 0, 0, 0, 0, 0)
(2000, 55127, 4, 22, 4, 0, 0, 0, 0, 0, 0)
(2000, 55129, 1, 9, 1, 0, 0, 0, 0, 0, 0)
(2000, 55131, 3, 11, 2, 0, 0, 0, 1, 0, 0)
(2000, 55133, 3, 26, 6, 0, 0, 0, 0, 0, 0)
(2000, 55135, 2, 10, 0, 0, 0, 0, 0, 0, 0)
(2000, 55137, 2, 5, 0, 0, 0, 0, 0, 0, 0)
(2000, 55139, 2, 10, 1, 0, 3, 0, 0, 0, 0)
(200

...

(2004, 48177, 0, 0, 17, 0, 0, 0, 0, 0, 0)
(2004, 48179, 0, 14, 0, 0, 0, 0, 0, 0, 0)
(2004, 48181, 2, 12, 0, 0, 0, 0, 0, 0, 0)
(2004, 48183, 0, 9, 0, 0, 0, 0, 0, 0, 0)
(2004, 48185, 0, 3, 3, 0, 0, 0, 1, 0, 0)
(2004, 48187, 0, 3, 16, 0, 0, 0, 1, 0, 0)
(2004, 48189, 2, 12, 0, 0, 0, 0, 0, 0, 0)
(2004, 48191, 0, 5, 0, 0, 0, 0, 0, 0, 0)
(2004, 48193, 1, 12, 4, 0, 0, 0, 0, 0, 0)
(2004, 48195, 1, 9, 3, 0, 0, 0, 0, 0, 0)
(2004, 48197, 2, 11, 0, 0, 0, 0, 0, 0, 0)
(2004, 48199, 0, 2, 0, 0, 0, 0, 2, 0, 0)
(2004, 48201, 1, 47, 5, 0, 2, 0, 11, 0, 0)
(2004, 48203, 0, 10, 0, 0, 0, 0, 0, 0, 0)
(2004, 48205, 2, 19, 4, 0, 0, 0, 2, 0, 0)
(2004, 48207, 1, 15, 1, 0, 0, 0, 6, 0, 0)
(2004, 48209, 0, 2, 21, 0, 0, 0, 0, 0, 0)
(2004, 48211, 1, 9, 0, 0, 0, 0, 0, 0, 0)
(2004, 48213, 1, 12, 2, 0, 0, 0, 0, 0, 0)
(2004, 48215, 1, 5, 2, 0, 0, 0, 0, 0, 0)
(2004, 48217, 1, 7, 5, 0, 0, 0, 0, 0, 0)
(2004, 48219, 1, 10, 1, 0, 0, 0, 0, 0, 0)
(2004, 48221, 1, 11, 2, 0, 0, 0, 0, 0, 0)
(2004, 48223, 1, 9, 0, 0, 0, 0, 0, 0, 0)


(2005, 5055, 0, 5, 0, 0, 0, 0, 0, 0, 0)
(2005, 5057, 0, 7, 0, 0, 0, 1, 0, 0, 0)
(2005, 5059, 0, 8, 0, 0, 0, 0, 1, 0, 0)
(2005, 5061, 1, 7, 0, 0, 0, 2, 0, 0, 0)
(2005, 5063, 0, 14, 0, 0, 0, 0, 0, 0, 0)
(2005, 5065, 0, 17, 0, 0, 0, 0, 1, 0, 0)
(2005, 5067, 0, 9, 0, 0, 0, 0, 0, 0, 0)
(2005, 5069, 0, 8, 2, 0, 0, 0, 2, 0, 0)
(2005, 5071, 0, 13, 1, 0, 0, 0, 0, 0, 0)
(2005, 5073, 0, 7, 0, 0, 0, 2, 0, 0, 0)
(2005, 5075, 0, 4, 0, 0, 0, 0, 0, 0, 0)
(2005, 5077, 0, 1, 0, 0, 0, 0, 0, 0, 0)
(2005, 5079, 0, 5, 0, 0, 0, 0, 0, 0, 0)
(2005, 5081, 0, 7, 0, 0, 0, 2, 0, 0, 0)
(2005, 5083, 0, 7, 1, 0, 0, 0, 0, 0, 0)
(2005, 5085, 0, 16, 0, 0, 0, 0, 2, 0, 0)
(2005, 5087, 1, 9, 1, 0, 0, 2, 0, 0, 0)
(2005, 5089, 0, 5, 2, 0, 0, 0, 0, 0, 0)
(2005, 5091, 0, 12, 0, 0, 0, 2, 0, 0, 2)
(2005, 5093, 0, 12, 0, 0, 0, 0, 1, 0, 0)
(2005, 5095, 0, 10, 1, 0, 0, 0, 0, 0, 0)
(2005, 5097, 0, 9, 0, 0, 0, 0, 2, 0, 0)
(2005, 5099, 0, 4, 0, 0, 0, 1, 0, 0, 0)
(2005, 5101, 0, 6, 1, 0, 0, 0, 0, 0, 0)
(2005, 5103, 0, 15, 2, 0, 0, 0, 0

(2005, 24025, 0, 6, 0, 0, 0, 0, 0, 0, 0)
(2005, 24027, 0, 6, 2, 0, 0, 0, 0, 0, 0)
(2005, 24029, 24, 14, 1, 0, 6, 2, 0, 0, 0)
(2005, 24031, 0, 14, 2, 0, 0, 0, 0, 0, 0)
(2005, 24033, 7, 14, 2, 0, 0, 0, 0, 0, 0)
(2005, 24035, 26, 14, 0, 0, 6, 2, 0, 0, 0)
(2005, 24037, 6, 1, 3, 0, 0, 0, 0, 0, 0)
(2005, 24039, 6, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 24041, 12, 10, 0, 0, 3, 1, 0, 0, 0)
(2005, 24043, 6, 7, 1, 0, 0, 0, 0, 0, 0)
(2005, 24045, 6, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 24047, 12, 1, 0, 0, 0, 0, 0, 0, 0)
(2005, 24510, 6, 9, 2, 0, 0, 0, 0, 0, 0)
(2005, 25001, 4, 16, 2, 0, 0, 0, 0, 0, 0)
(2005, 25003, 13, 15, 10, 0, 0, 0, 1, 0, 0)
(2005, 25005, 8, 19, 7, 0, 0, 0, 0, 0, 0)
(2005, 25007, 3, 6, 1, 0, 0, 0, 0, 0, 0)
(2005, 25009, 12, 32, 6, 0, 0, 0, 0, 0, 0)
(2005, 25011, 14, 14, 5, 0, 0, 0, 0, 0, 0)
(2005, 25013, 12, 25, 10, 0, 0, 0, 0, 0, 0)
(2005, 25015, 12, 9, 2, 0, 0, 0, 0, 0, 0)
(2005, 25017, 11, 28, 5, 0, 0, 0, 0, 0, 0)
(2005, 25019, 2, 8, 2, 0, 0, 0, 0, 0, 0)
(2005, 25021, 17, 52, 20, 0, 0, 0

(2005, 38045, 3, 8, 2, 0, 0, 0, 0, 0, 0)
(2005, 38047, 3, 7, 1, 0, 0, 0, 1, 0, 0)
(2005, 38049, 4, 7, 2, 0, 0, 0, 0, 0, 0)
(2005, 38051, 3, 6, 0, 0, 0, 0, 0, 0, 0)
(2005, 38053, 4, 6, 0, 0, 0, 0, 0, 0, 0)
(2005, 38055, 3, 8, 3, 0, 0, 0, 0, 0, 0)
(2005, 38057, 2, 11, 0, 0, 0, 0, 0, 0, 0)
(2005, 38059, 3, 25, 1, 0, 0, 0, 0, 0, 0)
(2005, 38061, 4, 15, 1, 0, 0, 0, 0, 0, 0)
(2005, 38063, 5, 27, 6, 0, 0, 0, 0, 0, 0)
(2005, 38065, 1, 7, 1, 0, 0, 0, 0, 0, 0)
(2005, 38067, 7, 22, 6, 0, 0, 0, 6, 0, 0)
(2005, 38069, 3, 5, 1, 0, 0, 0, 2, 0, 0)
(2005, 38071, 7, 31, 11, 0, 0, 0, 1, 0, 0)
(2005, 38073, 6, 17, 8, 0, 0, 0, 1, 0, 0)
(2005, 38075, 4, 6, 2, 0, 0, 0, 1, 0, 0)
(2005, 38077, 5, 38, 24, 0, 0, 0, 4, 0, 0)
(2005, 38079, 4, 9, 4, 0, 0, 0, 0, 0, 0)
(2005, 38081, 5, 16, 17, 0, 0, 0, 2, 0, 0)
(2005, 38083, 2, 6, 0, 0, 0, 0, 0, 0, 0)
(2005, 38085, 3, 5, 3, 0, 0, 0, 2, 0, 0)
(2005, 38087, 3, 6, 0, 0, 0, 0, 0, 0, 0)
(2005, 38089, 4, 10, 0, 0, 0, 0, 2, 0, 0)
(2005, 38091, 5, 18, 3, 0, 0, 0, 2, 0, 0)
(2

(2005, 51181, 4, 3, 0, 0, 0, 0, 1, 0, 0)
(2005, 51183, 5, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 51185, 2, 2, 0, 0, 0, 0, 0, 0, 0)
(2005, 51187, 3, 2, 2, 0, 0, 0, 0, 0, 0)
(2005, 51191, 3, 11, 0, 0, 0, 0, 0, 0, 0)
(2005, 51193, 18, 1, 0, 0, 0, 0, 0, 0, 0)
(2005, 51195, 2, 8, 1, 0, 0, 0, 0, 0, 0)
(2005, 51197, 2, 3, 0, 0, 0, 0, 0, 0, 0)
(2005, 51199, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 51510, 4, 2, 3, 0, 0, 0, 0, 0, 0)
(2005, 51515, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 51520, 3, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 51530, 4, 1, 1, 0, 0, 0, 0, 0, 0)
(2005, 51540, 6, 4, 5, 0, 0, 0, 0, 0, 0)
(2005, 51550, 3, 8, 3, 0, 0, 0, 0, 0, 0)
(2005, 51570, 0, 1, 0, 0, 0, 0, 0, 0, 0)
(2005, 51580, 1, 1, 0, 0, 0, 0, 0, 0, 0)
(2005, 51590, 3, 2, 1, 0, 0, 0, 0, 0, 0)
(2005, 51595, 4, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 51600, 8, 1, 5, 0, 0, 0, 0, 0, 0)
(2005, 51610, 4, 4, 3, 0, 0, 0, 0, 0, 0)
(2005, 51620, 4, 0, 0, 0, 0, 0, 0, 0, 0)
(2005, 51630, 6, 0, 2, 0, 0, 0, 0, 0, 0)
(2005, 51640, 3, 2, 1, 0, 0, 0, 0, 0, 0)
(2005, 51650, 

In [18]:
# Sample select testing 3
get_all_records_in_county_year_range = """ SELECT * FROM noaa_historical
WHERE fipscode=17033 AND year BETWEEN 2000 AND 2005;
"""
pg_cur.execute(get_all_records_in_county_year_range)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2000, 17033, 2, 5, 3, 0, 0, 0, 0, 0, 0)
(2001, 17033, 0, 8, 2, 0, 0, 0, 0, 0, 0)
(2002, 17033, 1, 8, 5, 0, 0, 0, 0, 0, 0)
(2003, 17033, 1, 8, 0, 0, 0, 0, 0, 0, 0)
(2004, 17033, 2, 8, 2, 0, 0, 0, 0, 0, 0)
(2005, 17033, 0, 2, 2, 0, 1, 0, 0, 0, 0)


In [19]:
# Sample select testing 4
get_all_records_in_county_specific_year = """ SELECT * FROM noaa_historical
WHERE fipscode=17033 AND year=2005;
"""
pg_cur.execute(get_all_records_in_county_specific_year)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(2005, 17033, 0, 2, 2, 0, 1, 0, 0, 0, 0)


### Creation of Danger Lable Table 

This table contains information about total number of events in each FIPS code (county) and assigned bins and corresponding labels generated for the bins. 

In [2]:
df_danger = pd.read_csv("https://raw.githubusercontent.com/labs12-should-i-live-here/DS/daniel-martinalarcon/data_output/6__danger/danger_labels.csv")

In [3]:
df_danger.head()

,FIPS,danger,bins_num,bins_labels
0,1001,337,2,Medium Risk
1,1003,467,3,High Risk
2,1005,213,2,Medium Risk
3,1007,221,2,Medium Risk
4,1009,445,3,High Risk


In [4]:
df_danger.dtypes

FIPS            int64
danger          int64
bins_num        int64
bins_labels    object
dtype: object

In [15]:
df_danger.shape

(3141, 4)

In [7]:
# Define the template of the table operation.
drop_risk_table = """DROP TABLE IF EXISTS risk_table;"""

# no primary key assigned since year or/and fipscode both
# repeat.
create_risk_table = """ CREATE TABLE risk_table(
    fipscode int,
    totalevents int,
    category int,
    categorylabel varchar(50)
);"""

# Get count of number of records in the table
get_count_risk_table = """SELECT count(*) FROM risk_table;"""

In [8]:
# Create the table and commit
pg_cur = pg_conn.cursor()

In [9]:
pg_cur.execute(drop_risk_table)
pg_cur.execute(create_risk_table)

In [10]:
pg_conn.commit()

In [11]:
pg_cur.execute(get_count_risk_table)
count = pg_cur.fetchone()
print(count)

(0,)


In [12]:
!wget https://raw.githubusercontent.com/labs12-should-i-live-here/DS/daniel-martinalarcon/data_output/6__danger/danger_labels.csv

--2019-05-15 09:49:50--  https://raw.githubusercontent.com/labs12-should-i-live-here/DS/daniel-martinalarcon/data_output/6__danger/danger_labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72606 (71K) [text/plain]
Saving to: ‘danger_labels.csv’

danger_labels.csv   100%[===================>]  70.90K  --.-KB/s    in 0.07s   

2019-05-15 09:49:51 (1.01 MB/s) - ‘danger_labels.csv’ saved [72606/72606]



In [13]:
with open('danger_labels.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f)  # Skip the header row.
    pg_cur.copy_from(f, 'risk_table', sep=',')

pg_conn.commit()

In [14]:
# Check the row count in noaa_historical after insert
pg_cur.execute(get_count_risk_table)
count = pg_cur.fetchone()
print(count)

(3141,)


In [16]:
# Sample select testing 1
all_fipscode = """ SELECT * FROM risk_table;
"""
pg_cur.execute(all_fipscode)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(1001, 337, 2, 'Medium Risk')
(1003, 467, 3, 'High Risk')
(1005, 213, 2, 'Medium Risk')
(1007, 221, 2, 'Medium Risk')
(1009, 445, 3, 'High Risk')
(1011, 153, 2, 'Medium Risk')
(1013, 142, 1, 'Low Risk')
(1015, 411, 3, 'High Risk')
(1017, 245, 2, 'Medium Risk')
(1019, 343, 2, 'Medium Risk')
(1021, 272, 2, 'Medium Risk')
(1023, 244, 2, 'Medium Risk')
(1025, 359, 2, 'Medium Risk')
(1027, 247, 2, 'Medium Risk')
(1029, 209, 2, 'Medium Risk')
(1031, 306, 2, 'Medium Risk')
(1033, 751, 3, 'High Risk')
(1035, 133, 1, 'Low Risk')
(1037, 174, 2, 'Medium Risk')
(1039, 280, 2, 'Medium Risk')
(1041, 145, 1, 'Low Risk')
(1043, 740, 3, 'High Risk')
(1045, 380, 2, 'Medium Risk')
(1047, 251, 2, 'Medium Risk')
(1049, 789, 3, 'High Risk')
(1051, 450, 3, 'High Risk')
(1053, 220, 2, 'Medium Risk')
(1055, 434, 3, 'High Risk')
(1057, 254, 2, 'Medium Risk')
(1059, 451, 3, 'High Risk')
(1061, 272, 2, 'Medium Risk')
(1063, 161, 2, 'Medium Risk')
(1065, 214, 2, 'Medium Risk')
(1067, 235, 2, 'Medium Risk')
(1069, 

(24047, 344, 2, 'Medium Risk')
(24510, 490, 3, 'High Risk')
(25001, 501, 3, 'High Risk')
(25003, 1087, 3, 'High Risk')
(25005, 810, 3, 'High Risk')
(25007, 209, 2, 'Medium Risk')
(25009, 956, 3, 'High Risk')
(25011, 823, 3, 'High Risk')
(25013, 971, 3, 'High Risk')
(25015, 741, 3, 'High Risk')
(25017, 1385, 4, 'Very High Risk')
(25019, 247, 2, 'Medium Risk')
(25021, 1624, 4, 'Very High Risk')
(25023, 1029, 3, 'High Risk')
(25025, 466, 3, 'High Risk')
(25027, 1240, 4, 'Very High Risk')
(26001, 154, 2, 'Medium Risk')
(26003, 472, 3, 'High Risk')
(26005, 380, 2, 'Medium Risk')
(26007, 159, 2, 'Medium Risk')
(26009, 247, 2, 'Medium Risk')
(26011, 141, 1, 'Low Risk')
(26013, 372, 2, 'Medium Risk')
(26015, 233, 2, 'Medium Risk')
(26017, 327, 2, 'Medium Risk')
(26019, 140, 1, 'Low Risk')
(26021, 393, 2, 'Medium Risk')
(26023, 283, 2, 'Medium Risk')
(26025, 267, 2, 'Medium Risk')
(26027, 293, 2, 'Medium Risk')
(26029, 63, 1, 'Low Risk')
(26031, 159, 2, 'Medium Risk')
(26033, 56, 1, 'Low Risk')

(47079, 285, 2, 'Medium Risk')
(47081, 335, 2, 'Medium Risk')
(47083, 183, 2, 'Medium Risk')
(47085, 250, 2, 'Medium Risk')
(47087, 213, 2, 'Medium Risk')
(47089, 270, 2, 'Medium Risk')
(47091, 188, 2, 'Medium Risk')
(47093, 526, 3, 'High Risk')
(47095, 173, 2, 'Medium Risk')
(47097, 246, 2, 'Medium Risk')
(47099, 509, 3, 'High Risk')
(47101, 194, 2, 'Medium Risk')
(47103, 438, 3, 'High Risk')
(47105, 286, 2, 'Medium Risk')
(47107, 377, 2, 'Medium Risk')
(47109, 235, 2, 'Medium Risk')
(47111, 242, 2, 'Medium Risk')
(47113, 403, 3, 'High Risk')
(47115, 293, 2, 'Medium Risk')
(47117, 231, 2, 'Medium Risk')
(47119, 425, 3, 'High Risk')
(47121, 189, 2, 'Medium Risk')
(47123, 449, 3, 'High Risk')
(47125, 451, 3, 'High Risk')
(47127, 264, 2, 'Medium Risk')
(47129, 287, 2, 'Medium Risk')
(47131, 266, 2, 'Medium Risk')
(47133, 286, 2, 'Medium Risk')
(47135, 202, 2, 'Medium Risk')
(47137, 146, 1, 'Low Risk')
(47139, 274, 2, 'Medium Risk')
(47141, 474, 3, 'High Risk')
(47143, 277, 2, 'Medium Ris

In [17]:
# Sample select category 2 
category_2 = """ SELECT * FROM risk_table
WHERE category = 2;
"""
pg_cur.execute(category_2)
rows = pg_cur.fetchall()

for row in rows:
    print(row)

(1001, 337, 2, 'Medium Risk')
(1005, 213, 2, 'Medium Risk')
(1007, 221, 2, 'Medium Risk')
(1011, 153, 2, 'Medium Risk')
(1017, 245, 2, 'Medium Risk')
(1019, 343, 2, 'Medium Risk')
(1021, 272, 2, 'Medium Risk')
(1023, 244, 2, 'Medium Risk')
(1025, 359, 2, 'Medium Risk')
(1027, 247, 2, 'Medium Risk')
(1029, 209, 2, 'Medium Risk')
(1031, 306, 2, 'Medium Risk')
(1037, 174, 2, 'Medium Risk')
(1039, 280, 2, 'Medium Risk')
(1045, 380, 2, 'Medium Risk')
(1047, 251, 2, 'Medium Risk')
(1053, 220, 2, 'Medium Risk')
(1057, 254, 2, 'Medium Risk')
(1061, 272, 2, 'Medium Risk')
(1063, 161, 2, 'Medium Risk')
(1065, 214, 2, 'Medium Risk')
(1067, 235, 2, 'Medium Risk')
(1081, 356, 2, 'Medium Risk')
(1085, 185, 2, 'Medium Risk')
(1087, 194, 2, 'Medium Risk')
(1091, 204, 2, 'Medium Risk')
(1099, 197, 2, 'Medium Risk')
(1105, 167, 2, 'Medium Risk')
(1107, 262, 2, 'Medium Risk')
(1109, 184, 2, 'Medium Risk')
(1111, 324, 2, 'Medium Risk')
(1113, 264, 2, 'Medium Risk')
(1115, 371, 2, 'Medium Risk')
(1119, 260

(37195, 264, 2, 'Medium Risk')
(37197, 312, 2, 'Medium Risk')
(38001, 314, 2, 'Medium Risk')
(38007, 249, 2, 'Medium Risk')
(38009, 327, 2, 'Medium Risk')
(38011, 337, 2, 'Medium Risk')
(38013, 272, 2, 'Medium Risk')
(38021, 322, 2, 'Medium Risk')
(38023, 227, 2, 'Medium Risk')
(38025, 345, 2, 'Medium Risk')
(38029, 317, 2, 'Medium Risk')
(38031, 263, 2, 'Medium Risk')
(38033, 283, 2, 'Medium Risk')
(38037, 315, 2, 'Medium Risk')
(38041, 313, 2, 'Medium Risk')
(38043, 291, 2, 'Medium Risk')
(38045, 303, 2, 'Medium Risk')
(38047, 246, 2, 'Medium Risk')
(38049, 368, 2, 'Medium Risk')
(38051, 288, 2, 'Medium Risk')
(38053, 379, 2, 'Medium Risk')
(38057, 323, 2, 'Medium Risk')
(38061, 378, 2, 'Medium Risk')
(38065, 248, 2, 'Medium Risk')
(38069, 255, 2, 'Medium Risk')
(38075, 262, 2, 'Medium Risk')
(38079, 239, 2, 'Medium Risk')
(38083, 208, 2, 'Medium Risk')
(38085, 285, 2, 'Medium Risk')
(38087, 245, 2, 'Medium Risk')
(38089, 386, 2, 'Medium Risk')
(38095, 402, 2, 'Medium Risk')
(38103, 